# Exploration and Exploitation

In [1]:
using PGFPlots
using Interact
using Reactive
using Distributions
include("helpers.jl")
include("bandits.jl");

WebIO._IJuliaInit()

## Multi-Armed Bandit Problems

In [2]:
using Random
Random.seed!(2)
arms = 3
b = Bandit(arms)
banditTrial(b)

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 1"))

Observable("0 wins out of 0 tries (NaN percent)")

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 2"))

Observable("0 wins out of 0 tries (NaN percent)")

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 3"))

Observable("0 wins out of 0 tries (NaN percent)")

(div { className="interact-widget" }
    (div { className="interact-flex-row interact-widget" }
    (div { className="interact-flex-row-left" }
      (label { className="interact " style=Dict{Any, Any}(:padding => "5px 10px 0px 10px") }
        "True Params"))
    (div { className="interact-flex-row-center" }
      (div { attributes="data-bind" => "foreach : options_js" className="buttons has-addons is-centered" }
        (span { attributes=Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}") }
          (label { attributes=Dict("data-bind" => "text : key") }))))
    (div { className="interact-flex-row-right" }
      (div))))

Observable("")

## Undirected Exploration Strategies

In [3]:
struct AlwaysPull <: BanditPolicy
    arm::Int
end

arm(b::AlwaysPull, s::BanditStatistics) = b.arm;

In [4]:
mutable struct ExploreThenCommit <: BanditPolicy
    k::Int
    commitment::Int
end
ExploreThenCommit(k) = ExploreThenCommit(k, 0)
function reset!(p::ExploreThenCommit)
    p.commitment = 0
    return nothing
end

function arm(b::ExploreThenCommit, s::BanditStatistics) 
    if sum(s.numTries) < b.k
        return rand(1:numArms(s))
    elseif sum(s.numTries) == b.k
        b.commitment = argmax(winProbabilities(s))
        return b.commitment
    else
        return b.commitment
    end
end;

In [5]:
struct EpsGreedy <: BanditPolicy
    eps::Real 
end

function arm(b::EpsGreedy, s::BanditStatistics)
    if rand() < b.eps
        return rand(1:numArms(s))
    else
        ps = winProbabilities(s)
        return rand(findall(ps .== maximum(ps))) # like argmax but breaks ties randomly
    end
end;

In [6]:
steps = 50
iterations = 10000
params = rand(Random.Xoshiro(1), 10)
display(params)
bandit = Bandit(params)
# bandit = Bandit(collect(1:-0.2:0.1))
@manipulate for epsgreedy in exp10.(-3:0.2:0), commit in 0:10:50
    policies = [
        "explore-commit" => ExploreThenCommit(commit),
        "eps greedy" => EpsGreedy(epsgreedy),
    ]
    curves = [Plots.Linear([1,steps], fill(maximum(params),2), legendentry="best arm", style="very thick, dashed", mark="none")]
    append!(curves, learningCurves(bandit, policies, steps=steps, iterations=iterations))
    Axis(curves, style="legend pos=south east", ymin=0, ymax=1, xmin=0, xmax=steps, xlabel="Pulls", ylabel="Average success", width="15cm", height="10cm")
end

10-element Vector{Float64}:
 0.0491718221481211
 0.11907881640750706
 0.3932710232252806
 0.024094310524527707
 0.6918572875342215
 0.7675180540873912
 0.08725304891274233
 0.8557176841095734
 0.8025607099234905
 0.661425351684768

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["epsgreedy"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 16, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(8), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/zach/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/home/zach/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/home/zach/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f300d2e0460, Task (runnable) @0x00007f300d2e0460), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.001\",\"0.00158489\",\"0.00251189\",\"0.00398107\",\"0.00630957\",\"0.01\",\"0.0158489\",\"0.0251189\",\"0.0398107\",\"0.0630957\",\"0.1\",\"0.158489\",\"0.251189\",\"0.398107\",\"0.630957\",\"1.0\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"3728920638118148036\",\"id\":\"13674753975117918257\",\"type\":\"observable\"}),\"in

## Directed (non-Bayesian) Exploration Strategies

In [7]:
struct SoftMax <: BanditPolicy
    precision::Float64 
end

function arm(b::SoftMax, s::BanditStatistics)
    p = exp.(b.precision * winProbabilities(s))
    p = p / sum(p)
    D = Categorical(p)
    return rand(D)
end;

In [8]:
struct UCB <: BanditPolicy
    c::Float64
end
    
function arm(p::UCB, s::BanditStatistics)
    logn = log(max(2, sum(s.numTries)))
    ucbs = winProbabilities(s) .+ p.c*sqrt.(logn./s.numTries)
    return rand(findall(ucbs .== maximum(ucbs))) # like argmax but breaks ties randomly
end    

arm (generic function with 5 methods)

In [9]:
steps = 100
iterations = 1000
params = rand(Random.Xoshiro(1), 10)
display(params)
bandit = Bandit(params)
# bandit = Bandit(collect(1:-0.2:0.1))
@manipulate for epsgreedy in exp10.(-3:0.2:0), softmax in 0:5:40, ucb_c in exp10.(range(-2, 0, 8))
    policies = [
        "softmax ($softmax)" => SoftMax(softmax),
        "eps greedy ($epsgreedy)" => EpsGreedy(epsgreedy),
        "ucb ($ucb_c)" => UCB(ucb_c),
    ]
    curves = [Plots.Linear([1,steps], fill(maximum(params),2), legendentry="best arm", style="very thick, dashed", mark="none")]
    append!(curves, learningCurves(bandit, policies, steps=steps, iterations=iterations))
    Axis(curves, style="legend pos=south east", ymin=0, ymax=1, xmin=0, xmax=steps, xlabel="Pulls", ylabel="Average success", width="15cm", height="10cm")
end

10-element Vector{Float64}:
 0.0491718221481211
 0.11907881640750706
 0.3932710232252806
 0.024094310524527707
 0.6918572875342215
 0.7675180540873912
 0.08725304891274233
 0.8557176841095734
 0.8025607099234905
 0.661425351684768

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["epsgreedy"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 16, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(8), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/zach/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/home/zach/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/home/zach/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f300d173c70, Task (runnable) @0x00007f300d173c70), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.001\",\"0.00158489\",\"0.00251189\",\"0.00398107\",\"0.00630957\",\"0.01\",\"0.0158489\",\"0.0251189\",\"0.0398107\",\"0.0630957\",\"0.1\",\"0.158489\",\"0.251189\",\"0.398107\",\"0.630957\",\"1.0\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"4217601758724328735\",\"id\":\"17161280497291908556\",\"type\":\"observable\"}),\"in

## Bayesian Model Estimation

In [10]:
Random.seed!(3)
arms = 2
b = Bandit(arms)
banditEstimation(b)

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 1"))

Observable("0 wins out of 0 tries (NaN percent)")

Observable(Axis(PGFPlots.Plots.Plot[PGFPlots.Plots.Linear(Real[0.0 0.010101010101010102 … 0.98989898989899 1.0; 1.0 1.0 … 1.0 1.0], "none", nothing, nothing, "Beta(1, 1)", nothing, nothing, nothing, false)], nothing, nothing, nothing, nothing, nothing, nothing, nothing, 0, 1, 0, nothing, nothing, nothing, nothing, nothing, nothing, "15cm", "10cm", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, "axis"))

(div { className="field interact-widget" }
    (button { attributes=Dict{Any, Any}("data-bind" => "click: function (){this.clicks(this.clicks()+1)}, css: {'is-loading' : loading}") className="is-medium button is-primary" style=Dict{String, Any}() }
    "Arm 2"))

Observable("0 wins out of 0 tries (NaN percent)")

Observable(Axis(PGFPlots.Plots.Plot[PGFPlots.Plots.Linear(Real[0.0 0.010101010101010102 … 0.98989898989899 1.0; 1.0 1.0 … 1.0 1.0], "none", nothing, nothing, "Beta(1, 1)", nothing, nothing, nothing, false)], nothing, nothing, nothing, nothing, nothing, nothing, nothing, 0, 1, 0, nothing, nothing, nothing, nothing, nothing, nothing, "15cm", "10cm", nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, nothing, "axis"))

(div { className="interact-widget" }
    (div { className="interact-flex-row interact-widget" }
    (div { className="interact-flex-row-left" }
      (label { className="interact " style=Dict{Any, Any}(:padding => "5px 10px 0px 10px") }
        "True Params"))
    (div { className="interact-flex-row-center" }
      (div { attributes="data-bind" => "foreach : options_js" className="buttons has-addons is-centered" }
        (span { attributes=Dict("data-bind" => "click: function (){\$root.update(val)}, css: {'is-primary is-selected' : \$root.index() == val, 'interact-widget is-medium button ' : true}") }
          (label { attributes=Dict("data-bind" => "text : key") }))))
    (div { className="interact-flex-row-right" }
      (div))))

Observable("")

In [11]:
# Select arm with highest alpha upper confidence bound
struct IntervalExploration <: BanditPolicy
    alpha::Real
end

function arm(b::IntervalExploration, s::BanditStatistics)
    qs = [quantile(Beta(s.numWins[i] + 1, s.numTries[i] - s.numWins[i] + 1), b.alpha) for i in 1:length(s.numWins)]
    return rand(findall(qs .== maximum(qs))) # like argmax but breaks ties randomly
end;

In [12]:
struct ThompsonSampling <: BanditPolicy end

function arm(p::ThompsonSampling, s::BanditStatistics)
    params = rand.([Beta(s.numWins[i] + 1, s.numTries[i] - s.numWins[i] + 1) for i in 1:length(s.numWins)])
    return argmax(params)
end

arm (generic function with 7 methods)

In [14]:
steps = 100
iterations = 1000
params = rand(Random.Xoshiro(1), 10)
display(params)
bandit = Bandit(params)
# bandit = Bandit(collect(1:-0.2:0.1))
@manipulate for epsgreedy in exp10.(-3:0.2:0), interval in 0.5:0.05:1, ucb_c in exp10.(range(-2, 0, 8))
    policies = [
        # "eps greedy ($epsgreedy)" => EpsGreedy(epsgreedy),
        # "ucb ($ucb_c)" => UCB(ucb_c),
        "interval" => IntervalExploration(interval),
        "thompson sampling" => ThompsonSampling(),
    ]
    curves = [Plots.Linear([1,steps], fill(maximum(params),2), legendentry="best arm", style="very thick, dashed", mark="none")]
    append!(curves, learningCurves(bandit, policies, steps=steps, iterations=iterations))
    Axis(curves, style="legend pos=south east", ymin=0, ymax=1, xmin=0, xmax=steps, xlabel="Pulls", ylabel="Average success", width="15cm", height="10cm")
end

10-element Vector{Float64}:
 0.0491718221481211
 0.11907881640750706
 0.3932710232252806
 0.024094310524527707
 0.6918572875342215
 0.7675180540873912
 0.08725304891274233
 0.8557176841095734
 0.8025607099234905
 0.661425351684768

Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Scope(Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :label), Any["epsgreedy"], Dict{Symbol, Any}(:className => "interact ", :style => Dict{Any, Any}(:padding => "5px 10px 0px 10px")))], Dict{Symbol, Any}(:className => "interact-flex-row-left")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :input), Any[], Dict{Symbol, Any}(:max => 16, :min => 1, :attributes => Dict{Any, Any}(:type => "range", Symbol("data-bind") => "numericValue: index, valueUpdate: 'input', event: {change: function (){this.changes(this.changes()+1)}}", "orient" => "horizontal"), :step => 1, :className => "slider slider is-fullwidth", :style => Dict{Any, Any}()))], Dict{Symbol, Any}(:className => "interact-flex-row-center")), Node{WebIO.DOM}(WebIO.DOM(:html, :div), Any[Node{WebIO.DOM}(WebIO.DOM(:html, :p), Any[], Dict{Symbol, Any}(:attributes => Dict("data-bind" => "text: formatted_val")))], Dict{Symbol, Any}(:className => "interact-flex-row-right"))], Dict{Symbol, Any}(:className => "interact-flex-row interact-widget")), Dict{String, Tuple{AbstractObservable, Union{Nothing, Bool}}}("changes" => (Observable(0), nothing), "index" => (Observable{Any}(8), nothing)), Set{String}(), nothing, Asset[Asset("js", "knockout", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout.js"), Asset("js", "knockout_punches", "/home/zach/.julia/packages/Knockout/HReiN/src/../assets/knockout_punches.js"), Asset("js", nothing, "/home/zach/.julia/packages/InteractBase/Qhvxg/src/../assets/all.js"), Asset("css", nothing, "/home/zach/.julia/packages/InteractBase/Qhvxg/src/../assets/style.css"), Asset("css", nothing, "/home/zach/.julia/packages/Interact/PENUy/src/../assets/bulma_confined.min.css")], Dict{Any, Any}("changes" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"changes\"]()) ? (this.valueFromJulia[\"changes\"]=true, this.model[\"changes\"](val)) : undefined})")], "index" => Any[WebIO.JSString("(function (val){return (val!=this.model[\"index\"]()) ? (this.valueFromJulia[\"index\"]=true, this.model[\"index\"](val)) : undefined})")]), WebIO.ConnectionPool(Channel{Any}(32), Set{AbstractConnection}(), Condition(Base.InvasiveLinkedList{Task}(Task (runnable) @0x00007f300c3cc180, Task (runnable) @0x00007f300c3cc180), Base.AlwaysLockedST(1))), WebIO.JSString[WebIO.JSString("function () {\n    var handler = (function (ko, koPunches) {\n    ko.punches.enableAll();\n    ko.bindingHandlers.numericValue = {\n        init: function(element, valueAccessor, allBindings, data, context) {\n            var stringified = ko.observable(ko.unwrap(valueAccessor()));\n            stringified.subscribe(function(value) {\n                var val = parseFloat(value);\n                if (!isNaN(val)) {\n                    valueAccessor()(val);\n                }\n            });\n            valueAccessor().subscribe(function(value) {\n                var str = JSON.stringify(value);\n                if ((str == \"0\") && ([\"-0\", \"-0.\"].indexOf(stringified()) >= 0))\n                     return;\n                 if ([\"null\", \"\"].indexOf(str) >= 0)\n                     return;\n                stringified(str);\n            });\n            ko.applyBindingsToNode(\n                element,\n                {\n                    value: stringified,\n                    valueUpdate: allBindings.get('valueUpdate'),\n                },\n                context,\n            );\n        }\n    };\n    var json_data = {\"formatted_vals\":[\"0.001\",\"0.00158489\",\"0.00251189\",\"0.00398107\",\"0.00630957\",\"0.01\",\"0.0158489\",\"0.0251189\",\"0.0398107\",\"0.0630957\",\"0.1\",\"0.158489\",\"0.251189\",\"0.398107\",\"0.630957\",\"1.0\"],\"changes\":WebIO.getval({\"name\":\"changes\",\"scope\":\"16564974260270695419\",\"id\":\"13919686351689267626\",\"type\":\"observable\"}),\"i